### ベクトルストアでの利用
1. FAISSに格納
2. FAISSから取得
3. FAISSでベクトル検索

In [1]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# from langchain_community.vectorstores import Pinecone # ほかのベクトルストアもコード変更なく同様に使える
from pypdf import PdfReader
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [4]:
# 1. PDFデータ読み込み
pdf_page = PdfReader("./data/llm.pdf")
text = ""
# PDFデータをテキストに変換
for page in pdf_page.pages:
    text += page.extract_text()
print(text[:100]) # 先頭100文字を表示

⼤規模⾔語モデル
出典 : フリー百科事典『ウィキペディア（ Wikipedia ）』
⼤規模⾔語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメ


In [5]:
# 2. データをチャンクに小分けにする
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # チャンクの最大文字数
    chunk_overlap=100   # チャンク間の重複文字数
)
docs = text_splitter.split_text(text) # テキストをチャンクに分割
docs_chunks = text_splitter.create_documents(docs) # チャンクをドキュメントに変換
print("Docサイズ", len(docs_chunks))
print("型", type(docs_chunks[0]))

Docサイズ 64
型 <class 'langchain_core.documents.base.Document'>


In [8]:
# 3. OpenAIのembeddingモデル取得
openAI_key = os.getenv('AZURE_OPENAI_API_KEY')
openAI_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

embeddings = None
if openAI_key != "" :
    embeddings = AzureOpenAIEmbeddings(
      api_key=openAI_key,
      azure_deployment="text-embedding-ada-002",
      # openai_api_versiton="2024-10-21",
      azure_endpoint=openAI_endpoint
    )
else:
    print("EmbeddingのAPIKeyを設定してください")

In [9]:
# 4. ベクトル化
# 一括でベクトル化する場合
# faiss_db = FAISS.from_documents(documents=docs, embedding=embeddings)

# プログレスバーを表示しながらベクトル化
faiss_db = None
with tqdm(total=len(docs_chunks), desc="documents 抽出") as pbar:
    for d in docs_chunks:
        if faiss_db:
            faiss_db.add_documents([d])
        else:
            faiss_db = FAISS.from_documents([d], embeddings)
        pbar.update(1)


documents 抽出: 100%|██████████| 64/64 [00:06<00:00,  9.68it/s]


In [10]:
# 5. ベクトル化データを保存
faiss_db.save_local("./db")

In [11]:
# ベクトル化データを読み込み
vectorstore  = FAISS.load_local(
    "./db", # ベクトル化データの保存先
    embeddings, # ベクトル化モデル
    allow_dangerous_deserialization=True # デシリアライズを許可 (デフォルトはFalse)でセキュリティを向上
)

# retrieverを取得
retriever = vectorstore.as_retriever()

In [12]:
# 類似文書の取得
similar_docs = retriever.get_relevant_documents("言語モデル", k=3)
# similar_docs = retriever.invoke("言語モデル", k=3)

# 類似文書を表示
for doc in similar_docs:
    print(doc)
    print("-----")

/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


page_content='⼤規模⾔語モデル
出典 : フリー百科事典『ウィキペディア（ Wikipedia ）』
⼤規模⾔語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千
万から数⼗億）を持つ⼈⼯ニューラルネットワークで構成されるコンピュータ⾔語モデルで、膨⼤なラベル
なしテキストを使⽤して⾃⼰教師あり学習または半教師あり学習によって訓練が⾏われる[1]。
LLM は 2018 年頃に登場し、さまざまなタスク（仕事）で優れた性能を発揮している。これにより、⾃然⾔語
処理の研究の焦点は、特定のタスクに特化した教師ありモデルを訓練するという以前のパラダイムから転換
した[2]。⼤規模⾔語モデルの応⽤は⽬覚ましい成果を上げているが、⼤規模⾔語モデルの開発はまだ始まっ
たばかりであり、多くの研究者が⼤規模⾔語モデルの改良に貢献している[3]。
⼤規模⾔語モデルという⽤語の正式な定義はないが、⼤規模コーパスで事前訓練された、数百万から数⼗億
以上のパラメータを持つディープラーニングモデルを指すことが多い。 LLM は、特定のタスク（感情分析、
固有表現抽出、数学的推論など）のために訓練されたものとは異なり、幅広いタスクに優れた汎⽤モデルで
ある[2][4]。 LLM がタスクを実⾏する能⼒や対応可能な範囲は、ある意味では設計における画期的な進歩には
依存せず、 LLM に費やされた資源（データ、パラメータサイズ、計算⼒）の量の関数であるように⾒え
る[5]。多数のパラメータを持ったニューラル⾔語モデルは、⽂の次の単語を予測するという単純なタスクで
⼗分に訓練することで、⼈間の⾔葉の構⽂や意味の多くを捉えられることがわかった。さらに、⼤規模な⾔
語モデルは、世の中に関するかなりの⼀般知識を⽰し、訓練中に⼤量の事実を「記憶」することができ
る[2]。
質の⾼い証拠とされる 2023 年のメタ分析によれば、⼤規模⾔語モデルの創造性に⽬を輝かせる研究者はもち
ろん世界中に存在し、⼩規模⾔語モデルにはできないタスクで⼤規模⾔語モデルが創造的であると主張する
学者もいるが、これは測定基準の選択によるものであり、創造性によるものではないことが⽰唆されてい
る。異なる測定基準を選択した場合、⼤規模⾔語モデルの創造性の優位性は